# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.362642     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.474475     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [22]:
model_all.coefficients.print_rows(num_rows=100)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.05595049576 |  None  |
|     bedrooms     |  None |        0.0         |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 8468.531086910105  |  None  |
|   sqft_living    |  None | 24.42072098244546  |  None  |
| sqft_living_sqrt |  None | 350.06055338605546 |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 842.0680348976231  |  None 

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [10]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate
import numpy as np

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [40]:
lowest_rss = np.inf
lowest_l1 = None
for l1 in np.logspace(1, 7, num=13):
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                    validation_set=None, l2_penalty=0., l1_penalty=l1, verbose=False)
    errors = model.predict(validation) - validation['price']
    RSS = np.dot(errors, errors)
    if RSS < lowest_rss:
        lowest_rss = RSS
        lowest_l1 = l1

print('lowest RSS was for L1 of : {}'.format(lowest_l1))
print('lowest RSS was: {}'.format(lowest_rss))

lowest RSS was for L1 of : 10.0
lowest RSS was: 625766285142460.6


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [33]:
l1 = 10
model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                    validation_set=None, l2_penalty=0., l1_penalty=l1, verbose=False)
print(model.coefficients.print_rows(num_rows=100))

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |   18993.4272127706   |  None  |
|     bedrooms     |  None |  7936.9676790313015  |  None  |
| bedrooms_square  |  None |  936.9933681932994   |  None  |
|    bathrooms     |  None |  25409.588934120668  |  None  |
|   sqft_living    |  None |  39.11513637970764   |  None  |
| sqft_living_sqrt |  None |  1124.650212807717   |  None  |
|     sqft_lot     |  None | 0.003483618222989654 |  None  |
|  sqft_lot_sqrt   |  None |  148.25839101140826  |  None  |
|      floors      |  None |  21204.335466950117  |  None  |
|  floors_square   |  None |  12915.524336072433  |  None  |
|    waterfront    |  None |   601905.594545272   |  None  |
|       view       |  None |  93312.85731187189   |  None  |
|    condition     |  None |  6609.035712447216   |  None  |
|      grade       |  No

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [41]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [42]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [45]:
lowest_rss = np.inf
lowest_l1 = None
models = []
for l1 in np.logspace(8, 10, num=20):
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                    validation_set=None, l2_penalty=0., l1_penalty=l1, verbose=False)
    errors = model.predict(validation) - validation['price']
    RSS = np.dot(errors, errors)
    non_zeros = model.coefficients['value'].nnz()
    
    m = {
        'name': l1,
        'non_zeros': non_zeros
    }
    models.append(m)
    if RSS < lowest_rss:
        lowest_rss = RSS
        lowest_l1 = l1

print('lowest RSS was for L1 of : {}'.format(lowest_l1))
print('lowest RSS was: {}'.format(lowest_rss))
models = sorted(models, key=lambda x: x['non_zeros'])
models

lowest RSS was for L1 of : 100000000.0
lowest RSS was: 627492659875075.4


[{'name': 7847599703.514623, 'non_zeros': 1},
 {'name': 10000000000.0, 'non_zeros': 1},
 {'name': 6158482110.6602545, 'non_zeros': 3},
 {'name': 4832930238.571753, 'non_zeros': 5},
 {'name': 3792690190.7322536, 'non_zeros': 6},
 {'name': 2976351441.6313133, 'non_zeros': 10},
 {'name': 2335721469.0901213, 'non_zeros': 12},
 {'name': 1832980710.8324375, 'non_zeros': 13},
 {'name': 1128837891.6846883, 'non_zeros': 15},
 {'name': 1438449888.2876658, 'non_zeros': 15},
 {'name': 885866790.4100832, 'non_zeros': 16},
 {'name': 263665089.87303555, 'non_zeros': 17},
 {'name': 335981828.6283788, 'non_zeros': 17},
 {'name': 428133239.8719396, 'non_zeros': 17},
 {'name': 545559478.1168514, 'non_zeros': 17},
 {'name': 695192796.1775591, 'non_zeros': 17},
 {'name': 100000000.0, 'non_zeros': 18},
 {'name': 127427498.57031322, 'non_zeros': 18},
 {'name': 162377673.91887242, 'non_zeros': 18},
 {'name': 206913808.111479, 'non_zeros': 18}]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [46]:
l1_penalty_min = 2976351441.6313133
l1_penalty_max = 3792690190.7322536

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [50]:
lowest_rss = np.inf
lowest_l1 = None
models = []
for l1 in np.linspace(l1_penalty_min, l1_penalty_max, num=20):
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                    validation_set=None, l2_penalty=0., l1_penalty=l1, verbose=False)
    errors = model.predict(validation) - validation['price']
    RSS = np.dot(errors, errors)
    non_zeros = model.coefficients['value'].nnz()
    
    m = {
        'name': l1,
        'non_zeros': non_zeros,
        'RSS': RSS,
        'model': model
    }
    models.append(m)
    if RSS < lowest_rss:
        lowest_rss = RSS
        lowest_l1 = l1

print('lowest RSS was for L1 of : {}'.format(lowest_l1))
print('lowest RSS was: {}'.format(lowest_rss))
models = sorted(models, key=lambda x: x['RSS'])
models

lowest RSS was for L1 of : 2976351441.6313133
lowest RSS was: 966925692362085.1


[{'name': 2976351441.6313133,
  'non_zeros': 10,
  'RSS': 966925692362085.1,
  'model': Class                          : LinearRegression
  
  Schema
  ------
  Number of coefficients         : 18
  Number of examples             : 9761
  Number of feature columns      : 17
  Number of unpacked features    : 17
  
  Hyperparameters
  ---------------
  L1 penalty                     : 2976351441.6313
  L2 penalty                     : 0.0
  
  Training Summary
  ----------------
  Solver                         : fista
  Solver iterations              : 10
  Solver status                  : Completed (Iteration limit reached).
  Training time (sec)            : 0.5031
  
  Settings
  --------
  Residual sum of squares        : 1096129858584939.0
  Training RMSE                  : 338444.2136
  
  Highest Positive Coefficients
  -----------------------------
  (intercept)                    : 196100.9378
  bathrooms                      : 17962.6967
  view                           : 113

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [48]:
3448968612.163437


3448968612.163437

In [57]:
models[11]['model'].coefficients.print_rows(num_rows=1000)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 222253.1925443279  |  None  |
|     bedrooms     |  None | 661.7227177822278  |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None |  15873.957259268   |  None  |
|   sqft_living    |  None | 32.41022145125974  |  None  |
| sqft_living_sqrt |  None | 690.1147733133166  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 2899.420269749878  |  None 